# Выгрузить все данные в xlsx в папку data


In [ ]:
import sqlite3
import pandas as pd

def read_db(name_db: str) -> dict:
    try:
        with sqlite3.connect(name_db) as conn:
            # Получаем названия всех таблиц
            tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
            name_tables = pd.read_sql(tables_query, conn)
            names_list = name_tables['name'].tolist()

            dataframes = {}
            for name in names_list:
                df = pd.read_sql(f'SELECT * FROM {name}', conn)  # Загружаем данные из таблицы и сохраняем в словаре
                dataframes[name] = df  # Сохраняем DataFrame в словаре
                df.to_excel(f'data/{name}.xlsx', index=False)  # Сохраняем в Excel

    except FileNotFoundError:
        print(f'Файл {name_db} не найден')
    except sqlite3.OperationalError as e:
        print(f'Ошибка подключения к базе данных: {e}')
    except pd.io.sql.DatabaseError as e:
        print(f'Ошибка выполнения SQL-запроса: {e}')
    except Exception as e:
        print(f'Произошла ошибка: {e}')

    return dataframes

def average_track_duration_by_genre(dataframes: dict) -> pd.DataFrame:
    if 'tracks' in dataframes:
        tracks_df = dataframes['tracks']
        
        # Проверяем, есть ли нужные столбцы
        if 'genre' in tracks_df.columns and 'milliseconds' in tracks_df.columns:
            # Группируем по жанрам и вычисляем среднюю длительность
            average_duration = tracks_df.groupby('genre')['milliseconds'].mean().reset_index()
            average_duration.columns = ['genre', 'average_duration_ms']  # Переименовываем столбцы
            return average_duration
        else:
            print("Не найдены необходимые столбцы 'genre' и 'milliseconds' в таблице 'tracks'.")
    else:
        print("Таблица 'tracks' не найдена в загруженных данных.")
    
    return None

# Пример использования
# db_name = 'your_database.db'  # Укажите имя вашей базы данных
dataframes = read_db('chinook.db')
average_duration_df = average_track_duration_by_genre(dataframes)

if average_duration_df is not None:
    print(average_duration_df)

Не найдены необходимые столбцы 'genre' и 'milliseconds' в таблице 'tracks'.


In [2]:
import sqlite3
import pandas as pd

with sqlite3.connect('chinook.db') as conn:
            # Получаем названия всех таблиц
            tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
            name_tables = pd.read_sql(tables_query, conn)
            names_list = name_tables['name'].tolist()

            dataframes = {}
            for name in names_list:
                df = pd.read_sql(f'SELECT * FROM {name}', conn)  # Загружаем данные из таблицы и сохраняем в словаре
                dataframes[name] = df  # Сохраняем DataFrame в словаре
                df.to_excel(f'data/{name}.xlsx', index=False)  # Сохраняем в Excel

#- Найдите среднюю длительность треков (Milliseconds) в каждом жанре (genres).
tracks_df = dataframes['tracks']
average_duration = tracks_df.groupby('GenreId')['Milliseconds'].mean().round(0).reset_index()
# print(average_duration)


# - Объедините таблицы tracks, albums и artists. Выведите список треков с названием альбома и именем исполнителя в xlsx
tracks_df = dataframes['tracks']
albums_df = dataframes['albums']
artists_df = dataframes['artists']

join_tracks_albums = pd.merge(tracks_df, albums_df, on='AlbumId', how='inner')

total_join = pd.merge(join_tracks_albums, artists_df, on='ArtistId', how='inner')
print(total_join)

      TrackId                                             Name_x  AlbumId  \
0           1            For Those About To Rock (We Salute You)        1   
1           2                                  Balls to the Wall        2   
2           3                                    Fast As a Shark        3   
3           4                                  Restless and Wild        3   
4           5                               Princess of the Dawn        3   
...       ...                                                ...      ...   
3498     3499  Pini Di Roma (Pinien Von Rom) \ I Pini Della V...      343   
3499     3500  String Quartet No. 12 in C Minor, D. 703 "Quar...      344   
3500     3501               L'orfeo, Act 3, Sinfonia (Orchestra)      345   
3501     3502  Quintet for Horn, Violin, 2 Violas, and Cello ...      346   
3502     3503                                      Koyaanisqatsi      347   

      MediaTypeId  GenreId                                           Compos

In [ ]:
total_join['full_nmae_track'] = total_join['Name_x'] + total_join['Title'] + total_join['Name_y']